# Setup

In [ ]:
import os,sys
%load_ext autoreload
%autoreload 2
# if the notebook is in a subdirectory
module_path = os.path.abspath(os.path.join('../'))
import pprint
import pandas as pd

# importlib.reload(Cli)
from lib.data.biglib import BigLib
from config.client_config import clientConfig

pp = pprint.PrettyPrinter(indent=4)

BigLib.configure(clientConfig)

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display
    display.max_columns = 1000
    display.max_rows = 1000
    display.max_colwidth = 199
    display.width = 1000
    # display.precision = 2  # set as needed

set_pandas_display_options()


In [ ]:
# from ipywidgets import Layout
# from ipysankeywidget import SankeyWidget
from tools.viz.sankey import Sankey
import numpy as np
import plotly.graph_objects as go
 
def draw_one(use_case=None, set_name=None, limit=50, sprint=None, where="", fname=None):
    where = f'dataset_display_name = "{set_name}" and use_case="{use_case}" '
    df = Sankey.query_paths(use_case=use_case, limit=limit, sprint=sprint, where=where)
    # remove self cycles
    # cycles = df[df.source == df.target].index
    # df = df.drop(cycles)

    fname = f"sp_{sprint}_{set_name}_{use_case}"
    # fname = fname or f'{where}-{use_case}-s{sprint}-d{limit}'
    base_path = f'./data/ignored/sankeys/{fname}'

    img_path = f'{base_path}.png'
    html_path = f'{base_path}.html'
    title = f'{where} | use_case: {use_case} | sprint: {sprint} | limit: {limit}'

    links = df.to_dict('records')

    df = pd.DataFrame(links)
    names = np.unique(df[["source","target"]], axis=None)
    nodes = pd.Series(index=names, data=range(len(names)))
    # df['label'] = nodes.index

    print('nodes\n', nodes)
    print('df\n', df.head(50))

    fig = go.Figure(
        go.Sankey(
            # node={"label": df['label']},
            node={"label": nodes.index},
            arrangement='snap',
            link={
                "source": nodes.loc[df["source"]],
                "target": nodes.loc[df["target"]],
                "value": df["value"],
                # "label": df["value"]
            },
        )
    )

    fig.update_layout(
        title_text=title, 
        font_size=14,
        width=1920, 
        height=1080
    )

    print('render to:', img_path)
    fig.write_image(img_path)
    fig.write_html(file=html_path)
    fig.show()
    return fig

set_name='sp24-XX-voice'
use_case='FLOWNAME'

print('set_name', set_name)

fig = draw_one(
    # where=where, 
    sprint=24,
    set_name=set_name,
    use_case=use_case,
    limit=100, 
    )


In [ ]:

cases = [
    'FLOW1',
    'FLOW2'
]

def loop():
    for sprint in [22]:
        for use_case in cases:
            print('sk:', use_case)
            sk = draw_one(use_case, limit=50, sprint=sprint)

loop()